In [24]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [80]:
app_df = pd.read_csv("../test_data/optimisation/appliances.csv")
cat_df = pd.read_csv("../test_data/optimisation/category.csv", converters={'possible_duration': literal_eval})
total_expected_consumption = 53000

In [86]:
cat_power_df = app_df.groupby("category", as_index=False).agg({"power":"sum", "name": "count"}).set_index("category")
duration_combinations_df = cat_df.set_index('id')[["possible_duration"]].transpose().explode("F").explode("A").explode("L").reset_index()
power = cat_power_df.loc[["A", "F", "L"], "power"].values
num_app = cat_power_df["name"].to_dict()
duration = duration_combinations_df[["A", "F", "L"]].values
ernergy = duration @ power
duration_combinations_df["energy"] = ernergy
optim_comb = duration_combinations_df.loc[duration_combinations_df.query("energy<=@total_expected_consumption")["energy"].idxmax()]
optim_energy = optim_comb["energy"]
optim_durations = optim_comb[["F", "A", "L"]].to_dict()
cat_df["duration"] = cat_df["id"].map(optim_durations)
cat_df["num_appliances"] = cat_df["id"].map(num_app)
cat_df["time_appliance"] = cat_df["duration"] / cat_df["num_appliances"]
cat_to_time = cat_df.set_index('id')["time_appliance"].to_dict()
app_df["time"] = app_df["category"].map(cat_to_time)
app_df["energy"] = app_df["power"] * app_df["time"]

In [87]:
cat_df

,id,possible_duration,minimum_duration,power_appliances,duration,num_appliances,time_appliance
0,F,"[6, 7, 8]",6.0,4500.0,8,2,4.000000
1,A,"[1, 2, 3, 4]",1.0,7000.0,1,3,0.333333
2,L,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...",6.0,1400.0,7,3,2.333333


In [88]:
app_df

,id,name,category,power,duration,consumption,time,energy
0,1,Fridge,F,2000.0,NaN,NaN,4.000000,8000.000000
1,2,Freezer,F,2500.0,NaN,NaN,4.000000,10000.000000
2,3,Washing machine,A,1500.0,NaN,NaN,0.333333,500.000000
3,4,Dishwasher,A,2500.0,NaN,NaN,0.333333,833.333333
4,5,Induction stove,A,3000.0,NaN,NaN,0.333333,1000.000000
5,6,TV,L,500.0,NaN,NaN,2.333333,1166.666667
6,7,Small light,L,100.0,NaN,NaN,2.333333,233.333333
7,8,Big light,L,800.0,NaN,NaN,2.333333,1866.666667
